<a href="https://colab.research.google.com/github/bugcaps/workjob/blob/main/2%EB%8B%A8%EA%B3%84_%EC%A3%BC%EC%86%8C%EC%A0%95%EB%B3%B4GEO%EC%A0%95%EB%B3%B4%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0_%EB%AC%B4%EB%A3%8Cver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 서울,경지 아파트의 주소 정보를 가지고 각 주소별 위경도 좌표를 조회하여 CSV에 생성한다.
* 무료로 좌표조회하여 생성하기
* 일부 누락되는 데이터가 있다.

In [ ]:
# 드라이브  연결
from google.colab import drive
drive.mount('/content/drvie') 

Mounted at /content/drvie


In [ ]:
!ls /content/drvie/MyDrive/apartHome/실거래가

 2019_21_서울경기_40_130.csv
'아파트(매매)__실거래가_2019_서울.csv'
'아파트(매매)__실거래가_2019_경기도.csv'
'아파트(매매)__실거래가_2020_서울.csv'
'아파트(매매)__실거래가_2020_경기도.csv'
'아파트(매매)__실거래가_2021_서울.csv'
'아파트(매매)__실거래가_2021_경기도.csv'
 2021_성동구.csv
 서울경기아파트주소.csv
 서울경기아파트주소_GEO.csv
 test.csv


In [ ]:
pip install  geopandas 

In [ ]:
import pandas as pd

address = pd.read_csv('/content/drvie/MyDrive/apartHome/실거래가/서울경기아파트주소.csv')
address

,주소
0,서울특별시 강남구 개포동 언주로 103
1,서울특별시 강남구 개포동 언주로 3
2,서울특별시 강남구 개포동 개포로 307
3,서울특별시 강남구 개포동 개포로 310
4,서울특별시 강남구 개포동 선릉로 7
...,...
13814,경기도 화성시 오산동 동탄감배산로 143
13815,경기도 화성시 오산동 동탄기흥로 393-20
13816,경기도 화성시 장안면 사곡리 밤밭일원길 131-10
13817,경기도 화성시 향남읍 장짐리 푸른들판로 251-8


In [ ]:
!pip install  geopandas 
!pip install geopy
#or in Colab: 
!pip install geocoder

In [ ]:
temp =address.sample(3)
address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13819 entries, 0 to 13818
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   주소      13819 non-null  object
dtypes: object(1)
memory usage: 108.1+ KB


pandas에 위도경도 좌표항목을 추가로 설정위해서 geocoder 추가
* !pip install geocoder 
* geocoder 를 사용하면 좌표값을 조회할 수있으나 일부 누락되는 부분이 있어서 이후에 추가로 구글 api를 사용하여 좌표조회가 필요하다.
* 그러나, geocoder는 호출비용이 들지 않는다. 



In [ ]:
import geocoder
#위도경도정보를 조회 좌표값을 분할 추출 한다.
# https://pypi.org/project/geocoder/
  
def geocoding(input_address):
  try:
   g = geocoder.osm(input_address)
   #g = geocoder.google(input_address, session=session) 
   if (g.status !='OK'):
     return 'NA','NA'
  except AttributeError:
    print("Problem with data or cannot Geocode.")
    return 'NA','NA'
  except GeocoderTimedOut:      
    print("Problem with data or cannot GeocoderTimedOut.")
  except TypeError:  
    return 'NA','NA'
  return g.osm['x'], g.osm['y']

## 진짜 모든 주소의 address
* 서울경기아파트주소_GEO.csv 생성

In [ ]:
import tqdm                                                                                                   
import numpy as np
import pandas as pd
import concurrent.futures
import multiprocessing
num_processes = multiprocessing.cpu_count()
display(num_processes)
df = pd.DataFrame({i: np.random.randint(1,100,size=1000) for i in ['a', 'b', 'c']})
 
# Define a function on the numbers
def func(a, b):
    return a+b
 
# Process the rows in chunks in parallel
with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    #df['result'] = list(pool.map(func, df['a'], df['b'], chunksize=10)) # Without a progressbar
    df['result'] = list(tqdm.tqdm(pool.map(func, df['a'], df['b'], chunksize=10), total=df.shape[0]))



2

100%|██████████| 1000/1000 [00:00<00:00, 32503.40it/s]


## Parallel processing pandas dataframes
* Sample test
* colab은  cpu가 2개다.

In [ ]:
pip install geocoder

### Test용 geocoder 호출

In [ ]:

temp= address.head(10)
temp

num_processes = multiprocessing.cpu_count()
import geopandas as gpd
# ## geo apikey 적용 데이터 조회
with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    #df['result'] = list(pool.map(func, df['a'], df['b'], chunksize=10)) # Without a progressbar
    temp['locations'] = list(tqdm.tqdm(pool.map(geocoding, temp['주소'], chunksize=50), total=temp.shape[0]))

temp[['lon','lat']] = pd.DataFrame(temp['locations'].tolist(),
                                 index=temp.index)
temp

In [ ]:
temp

,주소,locations,lon,lat
0,서울특별시 강남구 개포동 언주로 103,"(127.0528827, 37.481299500000006)",127.052883,37.481300
1,서울특별시 강남구 개포동 언주로 3,"(127.05343671789691, 37.483371950000006)",127.053437,37.483372
2,서울특별시 강남구 개포동 개포로 307,"(127.05575767360763, 37.4836782)",127.055758,37.483678
3,서울특별시 강남구 개포동 개포로 310,"(127.0570683, 37.4833807)",127.057068,37.483381
4,서울특별시 강남구 개포동 선릉로 7,"(127.0647353, 37.4792567)",127.064735,37.479257
5,서울특별시 강남구 개포동 삼성로 14,"(127.0666391, 37.4881031)",127.066639,37.488103
6,서울특별시 강남구 개포동 삼성로4길 17,"(127.06794538606707, 37.487307099999995)",127.067945,37.487307
7,서울특별시 강남구 개포동 개포로 516,"(127.0720025138362, 37.4888687)",127.072003,37.488869
8,서울특별시 강남구 개포동 언주로 105,"(127.05261245, 37.48162235)",127.052612,37.481622
9,서울특별시 강남구 개포동 언주로 110,"(127.05392779453325, 37.47929375)",127.053928,37.479294


#Thread를 활용한 주소정보 전체를 조회한다.
* 1000건씩 실행
* 13000건 조회시에 cpu가 2개라서 2시간 소요되었음. 
---



In [ ]:
num_processes = multiprocessing.cpu_count()

with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    #df['result'] = list(pool.map(func, df['a'], df['b'], chunksize=10)) # Without a progressbar
    address['locations'] = list(tqdm.tqdm(pool.map(geocoding, address['주소'], chunksize=50), total=address.shape[0]))



address[['lon','lat']] = pd.DataFrame(address['locations'].tolist(),
                                 index=address.index)
address.head()
address.to_csv('/content/drvie/MyDrive/apartHome/실거래가/서울경기아파트주소_GEO.csv',index = False)


In [ ]:
geoadds = pd.read_csv('/content/drvie/MyDrive/apartHome/실거래가/서울경기아파트주소_GEO.csv')
geoadds

,index,주소,locations,lon,lat
0,0,서울특별시 강남구 개포동 언주로 103,"(127.0528827, 37.481299500000006)",127.052883,37.481300
1,1,서울특별시 강남구 개포동 언주로 3,"(127.05343671789691, 37.483371950000006)",127.053437,37.483372
2,2,서울특별시 강남구 개포동 개포로 307,"(127.05575767360763, 37.4836782)",127.055758,37.483678
3,3,서울특별시 강남구 개포동 개포로 310,"(127.0570683, 37.4833807)",127.057068,37.483381
4,4,서울특별시 강남구 개포동 선릉로 7,"(127.0647353, 37.4792567)",127.064735,37.479257
...,...,...,...,...,...
13814,13814,경기도 화성시 오산동 동탄감배산로 143,"(127.0897249, 37.1958897)",127.089725,37.195890
13815,13815,경기도 화성시 오산동 동탄기흥로 393-20,"(127.0926572, 37.1866501)",127.092657,37.186650
13816,13816,경기도 화성시 장안면 사곡리 밤밭일원길 131-10,"('NA', 'NA')",NaN,NaN
13817,13817,경기도 화성시 향남읍 장짐리 푸른들판로 251-8,"(126.9110772, 37.1357985)",126.911077,37.135798
